# 运算符重载

## 基础知识

- 运算符重载让类拦截常规的python操作
- 类可重载所有python表达式运算符
- 类也可重载打印、函数调用、属性访问等内置运算
- 重载使类实例的行为更接近内置类型
- 重载是通过在一个类中提供特殊名称的方法来实现的

类提供了特殊名称的方法，当这个类的实例在对应的表达式中出现时，python会自动调用它们。

## 常用重载运算符

| 方法名 | 实现功能 | 触发调用的形式 |
| --- | --- | --- |
| `__init__` | 构造函数 | 对象创建 | 
| `__del__` | 析构函数 | x对象回收 | 
| `__add__` | "+"运算符 | X+Y，`__iadd__`是+= |
| `__or__` | "\|"运算符 | `__ior__`是\|= | 
| `__repr__` `__str__` | 打印，转换 | print(x) |
| `__call__` | 函数调用 | X() | 
| `__getattr__` | 属性访问 | x.undefine | 
| `__setattr__` | 属性赋值 | x.any = value |
| `__delattr__` | 属性删除 | del X.any |
| `__getattribute__` | 属性访问 | X.any |
| `__getitem__` | 索引，分片  | |
| `__setitem__` | 索引赋值和分片赋值 | x\[key\] = value |
| `__delitem__` | 索引删除和分片删除 | del x\[key\] |
| `__len__` | 长度 | len(X) |
| `__bool__` | 布尔测试 | bool(X) | 
| `__lt__` `__gt__` `__le__` `__ge__` `__eq__` `__ne__` | 比较 | |
| `__radd__` | 右侧+ | other+X |
| `__iadd__` | += | | 
| `__iter__`, `__next__` | 迭代上下文 | |
| `__contains__` | 成员关系测试 | item in X |
| `__index__` | 整数值转换 | | 
| `__enter__` `__exit__` | 上下文管理器 | | 
| `__get__` `__set__` `__delete__`| 描述符属性 | |
| `__new__` | 创建 | `__init__` 之前的对象创建 |  

## 索引和切片

In [3]:
class Indexer:
    def __init__(self):
        self.x = {}

    def __getitem__(self, key):
        return self.x[key]

    def __setitem__(self, key, value):
        self.x[key] = value

In [4]:
X = Indexer()
X['la'] = 33

In [6]:
X['la']

33

该方法用于分片时，技能对基础索引调用，有能被分片调用

In [8]:
class Indexer:
    data = [5, 6, 7, 8, 9]
    def __getitem__(self, index):
        print('getitem:', index)
        return self.data[index]

In [9]:
X = Indexer()
X[0]

getitem: 0


5

In [10]:
X[2:4]

getitem: slice(2, 4, None)


[7, 8]

## \_\_index\_\_

In [11]:
class C:
    def __index__(self):
        return 255

In [12]:
X = C()
hex(X) # __index__供内置的数字转换函数使用

'0xff'

## 索引迭代 \_\_getitem\_\_

In [21]:
 class StepperIndex:
     def __getitem__(self, i):
         return self.data[i]

In [22]:
X = StepperIndex()
X.data = 'spam'
X[1]

'p'

In [23]:
for item in X:
    print(item, end=' ')

s p a m

此外，成员关系测试in,列表推导，内置函数map，列表和元组赋值运算以及类型构造方法也都会自动调用`__getitem__`

## 可迭代对象 `__iter__` `__next__`

In [24]:
class Squares:
    def __init__(self, start, stop):
        self.value = start - 1
        self.stop = stop 
    
    def __iter__(self):
        return self 
        
    def __next__(self):
        if self.value == self.stop:
            raise StopIteration
        self.value += 1
        return self.value ** 2

In [25]:
for i in Squares(1, 5):
    print(i, end=' ')

1 4 9 16 25

In [26]:
X = Squares(1, 5)
I = iter(X)
next(I)

1

### 单个对象上的可迭代对象



In [30]:
class SkipObject:
    def __init__(self, wrapped):
        self.wrapped = wrapped
    
    def __iter__(self):
        # 每次调用迭代器都是返回一个新的待next方法的对象
        return SkipIterator(self.wrapped)

In [31]:
class SkipIterator:
    def __init__(self, wrapped):
        self.wrapped = wrapped
        self.offset = 0
    
    def __next__(self):
        if self.offset >= len(self.wrapped):
            raise StopIteration
        else:
            item = self.wrapped[self.offset]
            self.offset += 2
            return item 

In [32]:
alpha = 'abcdef'

In [45]:
skipper = SkipObject(alpha)
I = iter(skipper)
next(I)

'a'

In [44]:
skipper = SkipObject(alpha)
for x in skipper:
    for y in skipper:
        print(x+y, end=" ")

aa ac ae ca cc ce ea ec ee

## 成员关系 `__contains__` `__iter__` `__getitem__`

当 contains存在时，它将优先于iter，而iter则优先于getitem方法。contains方法应该把成员关系定义为对一个键值做映射，或者定义为对序列的搜索。下面是用三种方法实现对一个实例对象的成员关系测试和各种迭代上下文。

In [46]:
class Iters:
    def __init__(self, value):
        self.data = value 
    
    def __getitem__(self, i):
        print('get[%s]:' % i, end=" ")
        return self.data[i]

    def __iter__(self):
        print('iter=> ', end=" ")
        self.ix = 0
        return self 

    def __next__(self):
        print('next: ', end='')
        if self.ix == len(self.data):
            raise StopIteration
        item = self.data[self.ix]
        self.ix += 1
        return item 

    def __contains__(self, x):
        print('contains: ', end=' ')
        return x in self.data 

In [52]:
X = Iters([1, 2, 3, 4, 5])
print (3 in X) # 成员关系，这里使用的是contains

for i in X:
    print(i, end=' | ')

print()
print([i ** 2 for i in X])
print(list(map(bin, X)))

contains:  True
iter=>  next: 1 | next: 2 | next: 3 | next: 4 | next: 5 | next: 
iter=>  next: next: next: next: next: next: [1, 4, 9, 16, 25]
iter=>  next: next: next: next: next: next: ['0b1', '0b10', '0b11', '0b100', '0b101']


In [54]:
I = iter(X)
while True:
    try:
        print(next(I), end=' @ ')
    except StopIteration:
        break

iter=>  next: 1 @ next: 2 @ next: 3 @ next: 4 @ next: 5 @ next:

In [61]:
# 将contains去掉的话
class Iters2:
    def __init__(self, value):
        self.data = value 
    
    def __getitem__(self, i):
        print('get[%s]:' % i, end=" ")
        return self.data[i]

    def __iter__(self):
        print('iter=> next:', end=" ")
        for x in self.data:
            yield x 
            print('next:', end=' ')

In [70]:
x2 = Iters2([1, 2, 3, 4, 5])
print(3 in x2) # 原来优先使用contains的使用iter来实现了

for i in x2:
    print(i, end=' | ')
print()
print([i ** 2 for i in x2])
print(list(map(bin, x2)))

iter=> next: next: next: True
iter=> next: 1 | next: 2 | next: 3 | next: 4 | next: 5 | next: 
iter=> next: next: next: next: next: next: [1, 4, 9, 16, 25]
iter=> next: next: next: next: next: next: ['0b1', '0b10', '0b11', '0b100', '0b101']


In [67]:
# 将iter去掉的话
class Iters3:
    def __init__(self, value):
        self.data = value 
    
    def __getitem__(self, i):
        print('get[%s]:' % i, end=" ")
        return self.data[i]

In [71]:
x3 = Iters3([1, 2, 3, 4, 5])
print(3 in x3) # 现在全部用getitem实现了

for i in x3:
    print(i, end=' | ')
print()
print([i ** 2 for i in x3])
print(list(map(bin, x3)))

get[0]: get[1]: get[2]: True
get[0]: 1 | get[1]: 2 | get[2]: 3 | get[3]: 4 | get[4]: 5 | get[5]: 
get[0]: get[1]: get[2]: get[3]: get[4]: get[5]: [1, 4, 9, 16, 25]
get[0]: get[1]: get[2]: get[3]: get[4]: get[5]: ['0b1', '0b10', '0b11', '0b100', '0b101']


In [72]:
x3[2:]

get[slice(2, None, None)]:

[3, 4, 5]

## 属性访问： `__getattr__` `__setattr__`

### 属性引用

`__getattr__`方法用来拦截属性引用，当你用一个未定义的属性名称字符串对一个实例对象做点号运算时，它就会被调用。如果python能通过继承树搜索过程找到这个属性，那么该方法就不会被调用

In [73]:
class Empty:
    name = 'szq'
    def __getattr__(self, attrname):
        if attrname in ['age', 'name']:
            return 40
        else:
            raise AttributeError(attrname)

In [76]:
X = Empty()
print(X.age)
print(X.name)
X.address

40
szq


AttributeError: address

上例中，类本身只有一个name属性；
- 当使用点号求age时，由于没找到,所以会根据getattr计算出的值，作为该属性都得值
- 当求name时，因为继承树中存在，所以会以继承树为准
- 对于getattr中没有指定并且继承树中不存在adress，则发出异常

### 属性赋值和删除

setattr会拦截所有的属性赋值，但是在`setattr`中对任何`self.name`属性赋值，都将再次调用`setattr`,这潜在的导致了**无穷递归循环**。 为了避免循环，应该使用`self.__dict__['name']`

In [90]:
class Accesscontrol:
    name = 'szq'

    # 必须通过__dict__或者父类进行
    def __setattr__(self, attr, value):
        if attr == 'age':
            print('setattr: %s' % attr,  end=' ')
            self.__dict__[attr] = value + 10
        else:
            raise AttributeError(attr + " not allowed")
    
    def __getattr__(self, attr):
        if attr == 'age':
            return 40
        else:
            raise AttributeError(attr)

    def __delattr__(self, attr):
        print('del > %s' % attr)
        del self.__dict__[attr]

In [91]:
X = Accesscontrol()
X.age = 60
print()
print(X.age) # 通过setattr后，age已经存在于继承树中，不能再通过getattr了
del X.age # 使用重载的delattr删除age属性
print(X.age)
X.name = 'lqf' # setattr会拦截所有属性，包括继承树中的，所以该句不能执行

setattr: age 
70
del > age
40


AttributeError: name not allowed

其他属性管理工具：
- `__getattribute__`方法拦截所有的属性访问，而不只是那些未定义的，但是要小心循环调用
- `property`内置函数允许我们把方法对指定属性的访问和修改操作关联起来
- 描述符提供一个协议，被一个类的`__get__`和`__set__`方法与对指定类属性上的访问关联起来
- `slot`属性再类中声明，但在每个实例中都会创建隐式存储

## 字符串显式： `__repr__` `__str__`

In [92]:
class adder:
    def __init__(self, value=0):
        self.data = value 
    
    def __add__(self, other):
        self.data += other 

In [93]:
x = adder
print(x) # 此时打印x看不到它的心智，只能看到是一个类

<class '__main__.adder'>


In [104]:
class addrepr(adder):
    def __repr__(self):
        return 'addrepr(%s)' % self.data

    def __str__(self):
        return str(self.data) # 必须返回字符串，所以将int转为str

In [105]:
x = addrepr(2)
x + 1
print(x) # 定义了__repr__可以定制打印输出 调用的是 __str__
print([x]) # 内嵌到列表中时，使用的依然是repr
x  # 调用的是__repr__

3
[addrepr(3)]


addrepr(3)

- str 会首先被打印操作和str内置函数尝试，通常应该返回用户友好的
- repr 用于其他的场景，包括交互式命令行，repr函数等，以及没有`__str__`的情况, 以及内嵌
- 必须返回字符串

## 右侧加法和原位置加法： `__radd__` `__iadd__`

### 右侧加法

默认的add加法，不支持实例对象在+运算符右侧

In [106]:
class Adder:
    def __init__(self, value=0):
        self.data = value
    
    def __add__(self, other):
        return self.data + other

In [107]:
x = Adder(5)
x + 2

7

In [108]:
2 + x #不支持

TypeError: unsupported operand type(s) for +: 'int' and 'Adder'

In [111]:
class Adder:
    def __init__(self, value=0):
        self.data = value
    
    def __add__(self, other):
        return self.data + other
    
    def __radd__(self, other):
        print('radd > %d' %other ,end=' ')
        return self.__add__(other)

In [112]:
x = Adder(5)
2 + x # 重载radd后支持了

radd > 2

7

### 原位置加法

实现 `+=` 可以使用 iadd 和 add, 如果没有iadd则使用add, 但是通过iadd更高效

In [118]:
class Number:
    def __init__(self, val):
        self.val = val 
    
    def __iadd__(self, other):
        self.val += other 
        return self
    
    def __str__(self):
        return "{0}".format(self.val)

In [119]:
x = Number(5)
x += 1
print(x)

6


每个二元运算符都有类似的右侧和原位置重载方法，如减法，乘法等

## 调用表达式 `__call__`

允许类实例模拟类似函数的行为

In [121]:
class Prod:
    def __init__(self, value):
        self.data = value 
    
    def __call__(self, other):
        return self.data * other 

In [122]:
x = Prod(3)
x(5) # 3 * 5

15

## 比较运算

- 比较方法没有右侧变体
- 没有隐含关系， ==为真，不代表！=为假，两个都需要重载

In [125]:
class C:
    data = 'spam'
    def __gt__(self, other):
        return self.data > other 
    
    def __lt__(self, other):
        return self.data < other

In [126]:
x = C()
print( x < 'ham')
print( x > 'ham')

False
True


## 布尔测试 `__bool__` 和 `__len__`

python会首先尝试使用`__bool__`获取一个布尔值，如果不存在则会根据`__len__`长度确定值

In [127]:
class Truth:
    def __bool__(self):
        print('bool')
        return True

In [128]:
x = Truth()
if x :
    print('yes')

bool
yes


In [129]:
class Truth1:
    def __len__(self):
        print('len')
        return 5

In [130]:
x = Truth()
if x :
    print('yes')

bool
yes


In [131]:
class Truth2:
    pass 

In [132]:
# 都不存在时为真
x = Truth()
if x :
    print('yes') 

bool
yes


## 析构函数

In [154]:
class Life:
    def __init__(self, name='unknown'):
        print('hello' + name)
        self.name = name 
    
    def live(self):
        print(self.name)
    
    def __del__(self):
        print('goodbye' + self.name)

In [155]:
brain = Life('Brain')
brain.live()

helloBrain
Brain


In [156]:
brain = 2 # 重新赋值，会调用原来值得析构函数

goodbyeBrain
